# Prices and costs analyses
Routines to evaluate and visualize price and costs assumptions used as input for POMMES simulations

## Package imports

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

from pommesevaluation.tools import update_matplotlib_params

## Define data sets and global variables
* Define some model configuration defining which input to use
* Determine paths and file names for inputs and outputs to the script

In [ ]:
# Script configuration
LANGUAGE = "German"  # "German", "English"
is_renamed = False

# Model configuration in terms of prices and costs
fuel_cost_pathway = "NZE"
emissions_cost_pathway = "long-term"
inflation_rate = 0.02

# inputs and outputs
path_folder = "./model_inputs/pommesinvest/"
file_name_fuel_costs_ts = f"costs_fuel_{fuel_cost_pathway}_nominal_indexed_ts.csv"
file_name_emissions_costs_ts = f"costs_emissions_{emissions_cost_pathway}_nominal_indexed_ts.csv"

path_data_out = "./data_out/"
path_plots = "./plots/"
output_file = "fuel_and_emissions_costs"

# global variables
FUELS = {
    #"biomass": "#15b01a",
    #"uranium": "#e50000",
    #"lignite": "#7f2b0a",
    "hardcoal": "#000000",
    "natgas": "#ffd966",
    "hydrogen": "#6fa8dc",
    #"mixedfuels": "#a57e52",
    #"otherfossil": "#d8dcd6",
    #"waste": "#c04e01",
    "oil": "#aaa662",
}
FUELS_RENAMED = {
    "German": {
        "biomass": "Biomasse",
        "uranium": "Kernenergie",
        "lignite": "Braunkohle",
        "hardcoal": "Steinkohle",
        "natgas": "Erdgas",
        "hydrogen": "Wasserstoff",
        "mixedfuels": "gemischte Brennstoffe",
        "otherfossil": "sonstige Konventionelle",
        "waste": "Abfall",
        "oil": "Erdöl",
    },
    "English": {
        "biomass": "biomass",
        "uranium": "uranium",
        "lignite": "lignite",
        "hardcoal": "hard coal",
        "natgas": "natural gas",
        "hydrogen": "hydrogen",
        "mixedfuels": "mixed conventionals",
        "otherfossil": "other conventionals",
        "waste": "waste",
        "oil": "oil",
    },
}
EMISSION_COSTS_RENAMED = {
    "German": {
        "co2_price": "$CO_2$-Preis"
    },
    "English": {
        "co2_price": "$CO_2$ price"
    }
}

# Plotting config
PLOT_LABELS = {
    "German": {
        "x_label": "Jahr",
        "y_label": "Preis in $€_{2020}/MWh_{th}$",
        "secondary_y_label": "Preis in $€_{2020}/t CO_2$",
    },
    "English": {
        "x_label": "year",
        "y_label": "price in $€_{2020}/MWh_{th}$",
        "secondary_y_label": "price in $€_{2020}/t CO_2$",
    },
}

plt.rcParams.update({'font.size': 14})

In [ ]:
update_matplotlib_params(
    small_size=14, medium_size=16, large_size=18
)

## Read in data
* Read in **_nominal_** fuel and emissions prices data.
* Convert to **_real_** terms using the assumed inflation rate of 2% p.a.

In [ ]:
fuel_costs_nominal = pd.read_csv(f"{path_folder}{file_name_fuel_costs_ts}", index_col=0)
emissions_costs_nominal = pd.read_csv(f"{path_folder}{file_name_emissions_costs_ts}", index_col=0)

In [ ]:
# Filter values for DE only
fuel_costs_nominal = fuel_costs_nominal[
    [col for col in fuel_costs_nominal.columns if "DE_" in col]
]
emissions_costs_nominal = emissions_costs_nominal[
    [col for col in emissions_costs_nominal.columns if "DE_" in col]
]

fuel_costs_nominal.index = fuel_costs_nominal.index.str[:4].astype(int)
emissions_costs_nominal.index = emissions_costs_nominal.index.str[:4].astype(int)

# Convert from nominal to real terms
fuel_costs_real = fuel_costs_nominal.div(
    (1 + inflation_rate) ** (fuel_costs_nominal.index - 2020), axis=0
)
emissions_costs_real = emissions_costs_nominal.div(
    (1 + inflation_rate) ** (emissions_costs_nominal.index - 2020), axis=0
)

In [ ]:
# Rename
fuel_costs_real.rename(
    columns={col: col.rsplit("_")[-1] for col in fuel_costs_real.columns},
    inplace=True
)

# Emissions costs are the same for all energy carriers
emissions_costs_real = emissions_costs_real.loc[2020:2050, "DE_source_hardcoal"]
emissions_costs_real.name = "co2_price"

In [ ]:
# create a combined plot
# Legend handling has been improved by querying ChatGPT v3.5
fig, ax = plt.subplots(figsize=(10, 5))
if not is_renamed:
    fuel_costs_real = fuel_costs_real.loc[2020:2050, [fuel for fuel in FUELS]]
    fuel_costs_real.rename(columns=FUELS_RENAMED[LANGUAGE], inplace=True)
    color={
        FUELS_RENAMED[LANGUAGE][f]: FUELS[f] for f in FUELS_RENAMED[LANGUAGE] if f in FUELS
    }
    emissions_costs_real.rename(EMISSION_COSTS_RENAMED[LANGUAGE][emissions_costs_real.name], inplace=True)
    is_renamed = True
fuel_costs_real.to_csv(f"{path_data_out}{output_file}.csv")

_ = fuel_costs_real.loc[2020:2045].plot(kind="line", marker="o", ax=ax, color=color, legend=False)

ax2 = ax.twinx()

_ = emissions_costs_real.loc[2020:2045].plot(kind="line", marker="o", ax=ax2, color="#898989", legend=False)

_ = ax.set_xlabel(PLOT_LABELS[LANGUAGE]["x_label"])
_ = ax.set_ylabel(PLOT_LABELS[LANGUAGE]["y_label"])
_ = ax2.set_ylabel(PLOT_LABELS[LANGUAGE]["secondary_y_label"])

#_ = ax.legend(bbox_to_anchor=[-0.05, 0.5]) #[1.35, 1.05])
#_ = ax2.legend(bbox_to_anchor=[1.35, 0.7])
_ = ax.set_ylim([0, 200])
_ = ax2.set_ylim([0, 400])
#_ = ax.set_xticks(range(2020, int(max(fuel_costs_real.index + 1)), 1))
_ = ax.autoscale(enable=True, axis='x', tight=True)

handles, labels = [], []
for ax_object in [ax, ax2]:
    h, l = ax_object.get_legend_handles_labels()
    handles.extend(h)
    labels.extend(l)

_ = fig.legend(handles, labels, loc='upper center', bbox_to_anchor=[0.5, 0.0], fancybox=True, shadow=False, ncol=5)

ax.grid(color="#dedede")
fig.tight_layout()
fig.savefig(f"{path_plots}{output_file}.png", dpi=300, bbox_inches="tight")
plt.show()
plt.close()